# CS246 - Colab 2
## Frequent Pattern Mining in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [70]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u282-b08-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [71]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [72]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [75]:
sc = SparkContext.getOrCreate()
sc.stop()

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [76]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [78]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-04-13 17:19:35--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.203.100.2, 34.235.3.193, 54.85.41.146, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.203.100.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  14.06M  6.43MB/s    in 2.2s    

2021-04-13 17:19:38 (6.43 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [14746350/14746350]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [79]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [80]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [81]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [82]:
joined = products.join(orders, on = ['product_id'])
grouped = joined.groupBy('order_id').agg(collect_list("product_name"))
grouped.show()

+--------+--------------------------+
|order_id|collect_list(product_name)|
+--------+--------------------------+
|    1342|      [Bag of Organic B...|
|    1591|      [Cracked Wheat, O...|
|    4519|      [Beet Apple Carro...|
|    4935|                   [Vodka]|
|    6357|      [Fresh Mozzarella...|
|   10362|      [Crinkle Cut Fren...|
|   19204|      [Lemon Lime Thirs...|
|   29601|      [Toasted Coconut ...|
|   31035|      [Bag of Organic B...|
|   40011|      [Sea Salt Macadam...|
|   46266|      [Organic Uncured ...|
|   51607|      [Major Dickason's...|
|   58797|      [Bag of Organic B...|
|   61793|      [Raspberries, Nat...|
|   67089|      [Banana, Organic ...|
|   70863|      [Bathroom Tissue,...|
|   88674|      [Unsweetened Almo...|
|   91937|      [No. 485 Gin, Tra...|
|   92317|      [Banana, Indian M...|
|   99621|      [Organic Basil, F...|
+--------+--------------------------+
only showing top 20 rows



In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [83]:
from pyspark.ml.fpm import FPGrowth

In [85]:
fpGrowth = FPGrowth(itemsCol="collect_list(product_name)", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(grouped)

# Display frequent itemsets.
freqItemsets = model.freqItemsets

# Display generated association rules.
associationRules = model.associationRules

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
# model.transform(grouped).show()

Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [86]:
print("---------------------------------------")
print("Generated were")
print(str(freqItemsets.count()) + " frequent itemsets")
print("and")
print(str(associationRules.count()) + " association rules")
print("---------------------------------------")

---------------------------------------
Generated were
120 frequent itemsets
and
0 association rules
---------------------------------------


Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many frequent itemsets and association rules were generated.


In [87]:
fpGrowth2 = FPGrowth(itemsCol="collect_list(product_name)", minSupport=0.001, minConfidence=0.5)
model2 = fpGrowth2.fit(grouped)

# Display frequent itemsets.
freqItemsets2 = model2.freqItemsets

# Display generated association rules.
associationRules2 = model2.associationRules

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
# model2.transform(grouped).show()

+--------+--------------------------+----------+
|order_id|collect_list(product_name)|prediction|
+--------+--------------------------+----------+
|    1342|      [Bag of Organic B...|        []|
|    1591|      [Cracked Wheat, O...|        []|
|    4519|      [Beet Apple Carro...|        []|
|    4935|                   [Vodka]|        []|
|    6357|      [Fresh Mozzarella...|        []|
|   10362|      [Crinkle Cut Fren...|        []|
|   19204|      [Lemon Lime Thirs...|        []|
|   29601|      [Toasted Coconut ...|        []|
|   31035|      [Bag of Organic B...|        []|
|   40011|      [Sea Salt Macadam...|        []|
|   46266|      [Organic Uncured ...|        []|
|   51607|      [Major Dickason's...|        []|
|   58797|      [Bag of Organic B...|        []|
|   61793|      [Raspberries, Nat...|        []|
|   67089|      [Banana, Organic ...|        []|
|   70863|      [Bathroom Tissue,...|        []|
|   88674|      [Unsweetened Almo...|        []|
|   91937|      [No.

In [88]:
print("---------------------------------------")
print("Generated were")
print(str(freqItemsets2.count()) + " frequent itemsets")
print("and")
print(str(associationRules2.count()) + " association rules")
print("---------------------------------------")

---------------------------------------
Generated were
4444 frequent itemsets
and
11 association rules
---------------------------------------


To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.

In [102]:
sorted = freqItemsets2.sort(col("freq").desc())

In [133]:
sorted.show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Baby Spi...| 9784|
|       [Large Lemon]| 8135|
|   [Organic Avocado]| 7409|
|[Organic Hass Avo...| 7293|
|      [Strawberries]| 6494|
|             [Limes]| 6033|
|[Organic Raspberr...| 5546|
|[Organic Blueberr...| 4966|
|[Organic Whole Milk]| 4908|
|  [Organic Cucumber]| 4613|
|  [Organic Zucchini]| 4589|
|[Organic Yellow O...| 4290|
|    [Organic Garlic]| 4158|
|[Seedless Red Gra...| 4059|
|         [Asparagus]| 3868|
|[Organic Grape To...| 3823|
| [Organic Red Onion]| 3818|
+--------------------+-----+
only showing top 20 rows



In [134]:
associationRules2.show()

+--------------------+--------------------+------------------+------------------+--------------------+
|          antecedent|          consequent|        confidence|              lift|             support|
+--------------------+--------------------+------------------+------------------+--------------------+
|[Organic Kiwi, Or...|[Bag of Organic B...|0.5459770114942529| 4.627719489738336|0.001448071397541327|
|[Organic Raspberr...|[Bag of Organic B...|0.5984251968503937| 5.072272070642333|0.001737685677049...|
|[Organic Broccoli...|[Bag of Organic B...|0.5048231511254019| 4.278897986822536|0.001196564260073623|
|[Organic Unsweete...|[Bag of Organic B...|0.5141065830721003| 4.357584667849303|0.001249914258930...|
|[Yellow Onions, S...|            [Banana]|0.5357142857142857|3.7536332219526702|0.001143214261216...|
|[Organic Cucumber...|[Bag of Organic B...|          0.546875| 4.635330870478036|0.001066999977135...|
|[Organic Navel Or...|[Bag of Organic B...|0.5283018867924528| 4.47790453

In [135]:
associationRules2.collect()[2]

Row(antecedent=['Organic Broccoli', 'Organic Hass Avocado'], consequent=['Bag of Organic Bananas'], confidence=0.5048231511254019, lift=4.278897986822536, support=0.001196564260073623)

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!

## Results:
*   Q2.1: 120 
*   Q2.2: 0
*   Q2.3: 4444
*   Q2.4: 11
*   Q2.5: banana
*   Q2.6: 6494
*   Q2.7: 0.505
*   Q2.8: 4.279






